## 라이브러리

In [1]:
# GPU 메모리 할당 문제
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb

import numpy as np
import random
import gc
from tqdm import tqdm
from glob import glob

import xgboost as xgb
import catboost as cb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import log_loss
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import skew, kurtosis
from sklearn.manifold import TSNE
import missingno

## 데이터 전처리

In [3]:
# 데이터 불러오기

path = './data/'
train = pd.read_csv(path + 'train_features.csv')
train_label = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [21]:
# Pre-Processing Effect on the Accuracy of Event-Based Activity Segmentation and Classification through Inertial Sensors 
# https://www.researchgate.net/publication/281836367_Pre-Processing_Effect_on_the_Accuracy_of_Event-Based_Activity_Segmentation_and_Classification_through_Inertial_Sensors

train['acc_t']  = train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)
test['acc_t']  = test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)

In [5]:
# https://dacon.io/competitions/official/235689/codeshare/2375?page=1&dtype=recent&ptype=pub
# 간단한 LGBM으로 접근해보기

X_pivot_train = pd.pivot_table(data = train, # X_train의 데이터를 통해서
                               values = train.columns[2:],  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var'           # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = test, #
                               values = test.columns[2:], 
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

In [6]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

train_data = pd.merge(X_pivot_train, train_label.loc[:,['id','label']], on='id') # label_desc는 사용하지 않을 예정입니다.
train_data.label = train_data.label.astype('category')

display(train_data.head())

,id,sum_acc_t,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_t,mean_acc_x,...,std_gy_y,std_gy_z,var_acc_t,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z,label
0,0,631.801497,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,1.053002,0.931329,...,24.300479,25.275185,0.029707,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979,37
1,1,657.738745,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,1.096231,-0.766580,...,96.005289,75.545343,0.093854,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884,26
2,2,591.609160,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.986015,0.039836,...,118.956646,13.920337,0.017542,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778,3
3,3,579.795901,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,0.966327,-0.887702,...,36.953466,23.647153,0.010621,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841,26
4,4,663.331761,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,1.105553,-0.659018,...,60.514531,46.148326,0.087990,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017,26


In [7]:
# https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# 데이터 증강을 통해 과접합을 줄여보자 - DACON
# 원본 데이터 정리
import tensorflow as tf

x_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis].reshape(-1,600,7)

y_train = train_label['label']
y_categorical = tf.keras.utils.to_categorical(y_train)

x_test = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:,np.newaxis].reshape(-1,600,7)

100%|██████████████████████████████████████████████████████████████████████████████| 782/782 [00:00<00:00, 1724.61it/s]


In [8]:
# 푸리에 변환

def ft_trans(name,train,test):
    def train_test(check,num_col):

        
        if check =='train':
            df_checking=train.copy()
            train_datas = np.zeros((len(df_checking.id.unique()),304))
            
        elif check =='test':
            df_checking=test.copy()
            train_datas = np.zeros((len(df_checking.id.unique()),304))
                       

        for i,num in enumerate(tqdm(df_checking.id.unique())):

            tt = df_checking.loc[df_checking.id==num][name] -df_checking.loc[df_checking.id==num][name].mean()
            fmax = 50      # sampling frequency 1000 Hz
            dt = 1/fmax      # sampling period
            N  = 600      # length of signal

            t  = np.arange(0,N)*dt   # time = [0, dt, ..., (N-1)*dt]
            x = tt.values
            df = fmax/N   # df = 1/N = fmax/N
            f = np.arange(0,N)*df     #   frq = [0, df, ..., (N-1)*df]
            xf = np.fft.fft(x)*dt
            tq_index=f[0:int(N/2+1)]
            tq_abs= np.abs(xf[0:int(N/2+1)])

            results = pd.DataFrame(tq_abs,tq_index).reset_index().rename(columns={'index':'hz',0:'abs_value'})
            
            ar0 = np.array([num])
            ar1 =results.abs_value.values
            ar2 = np.array([skew(results.abs_value),kurtosis(results.abs_value, fisher=True)])
            return_value = np.concatenate([ar0,ar1 ,ar2])    
            train_datas[i] = return_value

        return train_datas

    
    col_ft = ['_'+str(x) for x in range(304)]
    
    num_col = len(col_ft)
    train_datas = train_test('train',num_col)
    test_datas = train_test('test',num_col)
    
    col_ft_F = ['id']+[name+"_"+x for x in col_ft[1:]]        
    train_df = pd.DataFrame(train_datas,columns= col_ft_F)
    test_df = pd.DataFrame(test_datas,columns= col_ft_F)
    
    train_df.id = train_df.id.astype('int')
    test_df.id = test_df.id.astype('int')
    
    
    return train_df ,test_df

# https://dacon.io/competitions/official/235689/codeshare/2374?page=1&dtype=recent&ptype=pub

In [9]:
train_fft,test_fft = ft_trans('acc_t',train,test)
train_fft.shape, test_fft.shape

100%|███████████████████████████████████████████████████████████████████████████████| 782/782 [00:01<00:00, 489.83it/s]


((3125, 304), (782, 304))

## 1번째 모델 : GRU

In [10]:
# 모델 1번: GRU

def build_gru(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))

    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    mc = tf.keras.callbacks.ModelCheckpoint(filepath='./model/lstm.h5',save_weights_only=True,monitor='loss',mode='min',save_best_only=True)
    
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in mskf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        #가벼운 모델 생성
        model = Sequential()
        model.add(GRU(32, input_shape=(train.shape[1],train.shape[2])))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(61, activation='softmax'))

        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(X,y,
                  epochs=50,
                  batch_size=64,
                  validation_data=(valid_x,valid_y),
                  callbacks=[es,mc]
                 )
        
        # save feat
        model.load_weights('./model/lstm.h5')
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

gru_train1, gru_test1 = build_gru(5, x_train, y_categorical, x_test, 1)
gru_train2, gru_test2 = build_gru(5, x_train, y_categorical, x_test, 2)

c:\users\pc\.conda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train on 2500 samples, validate on 625 samples
Epoch 1/50
2500/2500 [==============================] - 713s 285ms/sample - loss: 3.0842 - accuracy: 0.4352 - val_loss: 2.8129 - val_accuracy: 0.4848
Epoch 2/50
2500/2500 [==============================] - 1s 241us/sample - loss: 2.7747 - accuracy: 0.4860 - val_loss: 2.7602 - val_accuracy: 0.4848
Epoch 3/50
2500/2500 [==============================] - 1s 244us/sample - loss: 2.7519 - accuracy: 0.4860 - val_loss: 2.8170 - val_accuracy: 0.4848
Epoch 4/50
2500/2500 [==============================] - 1s 241us/sample - loss: 2.7267 - accuracy: 0.4860 - val_loss: 2.7221 - val_accuracy: 0.4864
Epoch 5/50
2500/2500 [==============================] - 1s 239us/sample - loss: 2.6799 - accuracy: 0.4912 - val_loss: 2.6907 - val_accuracy: 0.4912
Epoch 6/50
2500/2500 [==============================] - 1s 241us/sample - loss: 2.6203 - accuracy: 0.4980 - val_loss: 2.8205 - val_accuracy: 0.4992
Epoch 7/50
2500/2500 [==============================] - 1s 240u

Epoch 15/50
2500/2500 [==============================] - 1s 236us/sample - loss: 2.3004 - accuracy: 0.5272 - val_loss: 2.2670 - val_accuracy: 0.5408
Epoch 16/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.2896 - accuracy: 0.5288 - val_loss: 2.8610 - val_accuracy: 0.5152
Epoch 17/50
2500/2500 [==============================] - 1s 236us/sample - loss: 2.2994 - accuracy: 0.5300 - val_loss: 2.5296 - val_accuracy: 0.4992
Epoch 18/50
2500/2500 [==============================] - 1s 236us/sample - loss: 2.2780 - accuracy: 0.5316 - val_loss: 2.2643 - val_accuracy: 0.5392
Epoch 19/50
2500/2500 [==============================] - 1s 319us/sample - loss: 2.2707 - accuracy: 0.5296 - val_loss: 2.4659 - val_accuracy: 0.4928
Epoch 20/50
2500/2500 [==============================] - 1s 241us/sample - loss: 2.2606 - accuracy: 0.5300 - val_loss: 2.3619 - val_accuracy: 0.5424
Epoch 21/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.2681 - accuracy: 0.5316

2500/2500 [==============================] - 1s 238us/sample - loss: 2.3376 - accuracy: 0.5260 - val_loss: 2.3539 - val_accuracy: 0.5232
Epoch 13/50
2500/2500 [==============================] - 1s 239us/sample - loss: 2.3015 - accuracy: 0.5324 - val_loss: 2.5789 - val_accuracy: 0.4944
Epoch 14/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2936 - accuracy: 0.5312 - val_loss: 2.3307 - val_accuracy: 0.5280
Epoch 15/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2645 - accuracy: 0.5332 - val_loss: 2.3088 - val_accuracy: 0.5120
Epoch 16/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2411 - accuracy: 0.5340 - val_loss: 2.5406 - val_accuracy: 0.4832
Epoch 17/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.2245 - accuracy: 0.5356 - val_loss: 2.9739 - val_accuracy: 0.4576
Epoch 18/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2226 - accuracy: 0.5344 - val_loss:

c:\users\pc\.conda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Train on 2500 samples, validate on 625 samples
Epoch 1/50
2500/2500 [==============================] - 2s 643us/sample - loss: 3.1360 - accuracy: 0.4488 - val_loss: 2.7997 - val_accuracy: 0.4848
Epoch 2/50
2500/2500 [==============================] - 1s 244us/sample - loss: 2.7809 - accuracy: 0.4860 - val_loss: 2.7615 - val_accuracy: 0.4848
Epoch 3/50
2500/2500 [==============================] - 1s 243us/sample - loss: 2.7592 - accuracy: 0.4860 - val_loss: 2.7975 - val_accuracy: 0.4848
Epoch 4/50
2500/2500 [==============================] - 1s 243us/sample - loss: 2.7353 - accuracy: 0.4860 - val_loss: 2.7429 - val_accuracy: 0.4880
Epoch 5/50
2500/2500 [==============================] - 1s 242us/sample - loss: 2.6979 - accuracy: 0.4900 - val_loss: 2.6654 - val_accuracy: 0.4896
Epoch 6/50
2500/2500 [==============================] - 1s 242us/sample - loss: 2.6460 - accuracy: 0.4932 - val_loss: 2.6064 - val_accuracy: 0.4912
Epoch 7/50
2500/2500 [==============================] - 1s 243us/

2500/2500 [==============================] - 1s 237us/sample - loss: 2.1836 - accuracy: 0.5388 - val_loss: 2.2142 - val_accuracy: 0.5280
Epoch 36/50
2500/2500 [==============================] - 1s 244us/sample - loss: 2.1641 - accuracy: 0.5372 - val_loss: 2.3581 - val_accuracy: 0.5200
------------------
Train on 2500 samples, validate on 625 samples
Epoch 1/50
2500/2500 [==============================] - 2s 639us/sample - loss: 3.0406 - accuracy: 0.4508 - val_loss: 2.7765 - val_accuracy: 0.4864
Epoch 2/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.7771 - accuracy: 0.4856 - val_loss: 2.7502 - val_accuracy: 0.4864
Epoch 3/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.7514 - accuracy: 0.4856 - val_loss: 2.7557 - val_accuracy: 0.4864
Epoch 4/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.7146 - accuracy: 0.4856 - val_loss: 2.7175 - val_accuracy: 0.4864
Epoch 5/50
2500/2500 [==============================] - 

2500/2500 [==============================] - 1s 238us/sample - loss: 2.2800 - accuracy: 0.5324 - val_loss: 2.2674 - val_accuracy: 0.5344
Epoch 20/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2635 - accuracy: 0.5312 - val_loss: 2.2979 - val_accuracy: 0.5328
Epoch 21/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2658 - accuracy: 0.5332 - val_loss: 2.3166 - val_accuracy: 0.5152
Epoch 22/50
2500/2500 [==============================] - 1s 237us/sample - loss: 2.2576 - accuracy: 0.5308 - val_loss: 2.2609 - val_accuracy: 0.5312
Epoch 23/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2469 - accuracy: 0.5364 - val_loss: 2.2458 - val_accuracy: 0.5392
Epoch 24/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2342 - accuracy: 0.5368 - val_loss: 2.4997 - val_accuracy: 0.5184
Epoch 25/50
2500/2500 [==============================] - 1s 238us/sample - loss: 2.2439 - accuracy: 0.5308 - val_loss:

# 2번째 모델 : XGBoost

In [15]:
# 모델 2번: xgboost

def build_xgboost(split_num, train, target, test, rnd):
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 12,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                }
    
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_temp = xgb.DMatrix(valid_x)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        train_pred[val_idx] = model.predict(d_temp)
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

xgb_train1, xgb_test1 = build_xgboost(5, np.array(X_pivot_train.drop('id', axis=1)), np.array(y_train), np.array(X_pivot_test.drop('id', axis=1)), 1)
xgb_train2, xgb_test2 = build_xgboost(5, np.array(X_pivot_train.drop('id', axis=1)), np.array(y_train), np.array(X_pivot_test.drop('id', axis=1)), 2)
xgb_train3, xgb_test3 = build_xgboost(5, np.array(train_fft), np.array(y_train), np.array(test_fft), 1)
xgb_train4, xgb_test4 = build_xgboost(5, np.array(train_fft), np.array(y_train), np.array(test_fft), 2)

[0]	train-mlogloss:3.57991	valid-mlogloss:3.60734
[100]	train-mlogloss:0.21826	valid-mlogloss:1.01445
[200]	train-mlogloss:0.05776	valid-mlogloss:0.89880
[300]	train-mlogloss:0.03253	valid-mlogloss:0.88006
[400]	train-mlogloss:0.02457	valid-mlogloss:0.87348
[500]	train-mlogloss:0.02099	valid-mlogloss:0.87175
[600]	train-mlogloss:0.01903	valid-mlogloss:0.87098
[626]	train-mlogloss:0.01868	valid-mlogloss:0.87121
------------------
[0]	train-mlogloss:3.56618	valid-mlogloss:3.60424
[100]	train-mlogloss:0.21256	valid-mlogloss:1.09214
[200]	train-mlogloss:0.05654	valid-mlogloss:0.98386
[300]	train-mlogloss:0.03214	valid-mlogloss:0.96495
[400]	train-mlogloss:0.02447	valid-mlogloss:0.95708
[500]	train-mlogloss:0.02097	valid-mlogloss:0.95175
[592]	train-mlogloss:0.01917	valid-mlogloss:0.95146
------------------
[0]	train-mlogloss:3.56769	valid-mlogloss:3.58479
[100]	train-mlogloss:0.21601	valid-mlogloss:0.97419
[200]	train-mlogloss:0.05735	valid-mlogloss:0.87571
[300]	train-mlogloss:0.03248	val

In [20]:
sample_submssion = pd.read_csv(path + 'sample_submission.csv')
sample_submssion.iloc[:,1:] = xgb_test1
sample_submssion.to_csv("xgboost.csv", index = False)

## 3번째 모델 : CATBoost

In [16]:
# 모델 3번: catboost

def build_catboost(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train.iloc[train_idx]
        y = target.iloc[train_idx]
        valid_x = train.iloc[val_idx]
        valid_y = target.iloc[val_idx]

        model = cb.CatBoostClassifier(iterations=1000,
                                      learning_rate=0.01,
                                      l2_leaf_reg=3.5,
                                      depth=6,
                                      loss_function= 'MultiClass',
                                      eval_metric='AUC',
                                      use_best_model=True,
                                      random_seed=42,
                                      verbose=50,
                                      task_type="GPU")

        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds=30)
        
        # save feat
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

catboost_train1, catboost_test1 = build_catboost(5, X_pivot_train, y_train, X_pivot_test, 1)
catboost_train2, catboost_test2 = build_catboost(5, X_pivot_train, y_train, X_pivot_test, 2)
catboost_train3, catboost_test3 = build_catboost(5, train_fft, y_train, test_fft, 1)
catboost_train4, catboost_test4 = build_catboost(5, train_fft, y_train, test_fft, 2)

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8924458	best: 0.8924458 (0)	total: 52.6ms	remaining: 52.5s
50:	test: 0.9650582	best: 0.9650582 (50)	total: 2.54s	remaining: 47.4s
100:	test: 0.9738993	best: 0.9738993 (100)	total: 5.1s	remaining: 45.4s
150:	test: 0.9803245	best: 0.9803245 (150)	total: 7.64s	remaining: 42.9s
200:	test: 0.9842218	best: 0.9842218 (200)	total: 10.2s	remaining: 40.6s
250:	test: 0.9860862	best: 0.9861705 (247)	total: 12.8s	remaining: 38.2s
300:	test: 0.9873952	best: 0.9873952 (300)	total: 15.4s	remaining: 35.8s
350:	test: 0.9888541	best: 0.9888541 (350)	total: 18s	remaining: 33.3s
400:	test: 0.9895124	best: 0.9895535 (398)	total: 20.6s	remaining: 30.8s
450:	test: 0.9902556	best: 0.9902556 (450)	total: 23.2s	remaining: 28.3s
500:	test: 0.9907162	best: 0.9907162 (500)	total: 25.8s	remaining: 25.7s
550:	test: 0.9908260	best: 0.9909202 (542)	total: 28.5s	remaining: 23.2s
600:	test: 0.9914189	best: 0.9914285 (594)	total: 31.1s	remaining: 20.6s
650:	test: 0.9918542	best: 0.9918746 (641)	total: 33.7s	rem

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8875437	best: 0.8875437 (0)	total: 51.9ms	remaining: 51.8s
50:	test: 0.9622506	best: 0.9622506 (50)	total: 2.51s	remaining: 46.7s
100:	test: 0.9677880	best: 0.9679554 (99)	total: 5.03s	remaining: 44.8s
150:	test: 0.9727521	best: 0.9728459 (149)	total: 7.55s	remaining: 42.4s
200:	test: 0.9761931	best: 0.9761931 (200)	total: 10.1s	remaining: 40.1s
250:	test: 0.9784841	best: 0.9784939 (249)	total: 12.6s	remaining: 37.7s
300:	test: 0.9806446	best: 0.9806446 (300)	total: 15.2s	remaining: 35.3s
350:	test: 0.9813734	best: 0.9816722 (346)	total: 17.8s	remaining: 32.9s
400:	test: 0.9837399	best: 0.9837399 (400)	total: 20.4s	remaining: 30.5s
450:	test: 0.9846054	best: 0.9846665 (448)	total: 23s	remaining: 28s
500:	test: 0.9854098	best: 0.9854098 (500)	total: 25.6s	remaining: 25.5s
550:	test: 0.9860404	best: 0.9861243 (547)	total: 28.2s	remaining: 23s
600:	test: 0.9863744	best: 0.9863886 (595)	total: 30.8s	remaining: 20.4s
650:	test: 0.9868783	best: 0.9869188 (645)	total: 33.4s	remaini

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.8470540	best: 0.8470540 (0)	total: 70ms	remaining: 1m 9s
50:	test: 0.9249712	best: 0.9249712 (50)	total: 3.45s	remaining: 1m 4s
100:	test: 0.9379566	best: 0.9382761 (97)	total: 6.81s	remaining: 1m
150:	test: 0.9446923	best: 0.9449364 (146)	total: 10.2s	remaining: 57.6s
200:	test: 0.9471905	best: 0.9471905 (200)	total: 13.8s	remaining: 54.7s
250:	test: 0.9490075	best: 0.9493188 (241)	total: 17.3s	remaining: 51.6s
300:	test: 0.9512508	best: 0.9513760 (292)	total: 20.8s	remaining: 48.3s
350:	test: 0.9529442	best: 0.9529857 (349)	total: 24.3s	remaining: 44.9s
400:	test: 0.9537243	best: 0.9537553 (397)	total: 27.8s	remaining: 41.5s
450:	test: 0.9553467	best: 0.9553467 (450)	total: 31.3s	remaining: 38s
500:	test: 0.9565066	best: 0.9565909 (499)	total: 34.7s	remaining: 34.6s
550:	test: 0.9576662	best: 0.9577357 (536)	total: 38.2s	remaining: 31.1s
600:	test: 0.9592227	best: 0.9592227 (600)	total: 41.7s	remaining: 27.7s
650:	test: 0.9596165	best: 0.9598239 (641)	total: 45.2s	remainin

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8261992	best: 0.8261992 (0)	total: 68.3ms	remaining: 1m 8s
50:	test: 0.9142809	best: 0.9151484 (46)	total: 3.45s	remaining: 1m 4s
100:	test: 0.9241988	best: 0.9241988 (100)	total: 6.82s	remaining: 1m
150:	test: 0.9327394	best: 0.9327394 (150)	total: 10.2s	remaining: 57.6s
200:	test: 0.9393589	best: 0.9394086 (198)	total: 13.7s	remaining: 54.4s
250:	test: 0.9425398	best: 0.9425398 (250)	total: 17.2s	remaining: 51.4s
300:	test: 0.9433780	best: 0.9433780 (300)	total: 20.7s	remaining: 48.1s
350:	test: 0.9445608	best: 0.9446586 (344)	total: 24.2s	remaining: 44.8s
400:	test: 0.9457254	best: 0.9457371 (399)	total: 27.7s	remaining: 41.3s
450:	test: 0.9470171	best: 0.9470171 (450)	total: 31.1s	remaining: 37.9s
500:	test: 0.9475443	best: 0.9475985 (485)	total: 34.6s	remaining: 34.5s
550:	test: 0.9497635	best: 0.9497635 (550)	total: 38.1s	remaining: 31.1s
600:	test: 0.9508769	best: 0.9510985 (587)	total: 41.6s	remaining: 27.6s
650:	test: 0.9530180	best: 0.9530180 (650)	total: 45.1s	rem

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8223100	best: 0.8223100 (0)	total: 71.5ms	remaining: 1m 11s
50:	test: 0.9185141	best: 0.9190791 (49)	total: 3.47s	remaining: 1m 4s
100:	test: 0.9224327	best: 0.9225167 (99)	total: 6.87s	remaining: 1m 1s
150:	test: 0.9274679	best: 0.9274679 (150)	total: 10.3s	remaining: 58.1s
200:	test: 0.9324874	best: 0.9324874 (200)	total: 13.9s	remaining: 55.2s
250:	test: 0.9363255	best: 0.9363255 (250)	total: 17.4s	remaining: 51.9s
300:	test: 0.9374430	best: 0.9374430 (300)	total: 21s	remaining: 48.7s
350:	test: 0.9395526	best: 0.9395526 (350)	total: 24.5s	remaining: 45.3s
400:	test: 0.9421952	best: 0.9422032 (396)	total: 28s	remaining: 41.9s
450:	test: 0.9443457	best: 0.9443457 (450)	total: 31.6s	remaining: 38.4s
500:	test: 0.9460485	best: 0.9460485 (500)	total: 35.1s	remaining: 34.9s
550:	test: 0.9478192	best: 0.9478192 (550)	total: 38.5s	remaining: 31.4s
600:	test: 0.9488571	best: 0.9489168 (599)	total: 42s	remaining: 27.9s
650:	test: 0.9500364	best: 0.9501063 (648)	total: 45.5s	remain

## 4번째 모델 : Transformer

## Ensemble

In [17]:
xgb1_onehot = np.argmax(xgb_train1, axis=1).reshape(-1,1)
xgb2_onehot = np.argmax(xgb_train2, axis=1).reshape(-1,1)
xgb3_onehot = np.argmax(xgb_train3, axis=1).reshape(-1,1)
xgb4_onehot = np.argmax(xgb_train4, axis=1).reshape(-1,1)
catboost1_onehot = np.argmax(catboost_train1, axis=1).reshape(-1,1)
catboost2_onehot = np.argmax(catboost_train2, axis=1).reshape(-1,1)
catboost3_onehot = np.argmax(catboost_train3, axis=1).reshape(-1,1)
catboost4_onehot = np.argmax(catboost_train4, axis=1).reshape(-1,1)
gru1_onehot = np.argmax(gru_train1, axis=1).reshape(-1,1)
gru2_onehot = np.argmax(gru_train2, axis=1).reshape(-1,1)
# transformer1_onehot = np.argmax(transformer_train1, axis=1).reshape(-1,1)
# transformer2_onehot = np.argmax(transformer_train2, axis=1).reshape(-1,1)

xgb1_onehot_test = np.argmax(xgb_test1, axis=1).reshape(-1,1)
xgb2_onehot_test = np.argmax(xgb_test2, axis=1).reshape(-1,1)
xgb3_onehot_test = np.argmax(xgb_test3, axis=1).reshape(-1,1)
xgb4_onehot_test = np.argmax(xgb_test4, axis=1).reshape(-1,1)
catboost1_onehot_test = np.argmax(catboost_test1, axis=1).reshape(-1,1)
catboost2_onehot_test = np.argmax(catboost_test2, axis=1).reshape(-1,1)
catboost3_onehot_test = np.argmax(catboost_test3, axis=1).reshape(-1,1)
catboost4_onehot_test = np.argmax(catboost_test4, axis=1).reshape(-1,1)
gru1_onehot_test = np.argmax(gru_test1, axis=1).reshape(-1,1)
gru2_onehot_test = np.argmax(gru_test2, axis=1).reshape(-1,1)
# transformer1_onehot_test = np.argmax(transformer_test1, axis=1).reshape(-1,1)
# transformer2_onehot_test = np.argmax(transformer_test2, axis=1).reshape(-1,1)

train_final = np.hstack([xgb1_onehot, xgb2_onehot,
                         xgb3_onehot, xgb4_onehot,
                         catboost1_onehot, catboost2_onehot,
                         catboost3_onehot, catboost4_onehot,
                         gru1_onehot, gru2_onehot])
#                          transformer1_onehot, transformer2_onehot])

test_final = np.hstack([xgb1_onehot_test, xgb2_onehot_test,
                        xgb3_onehot_test, xgb4_onehot_test,
                        catboost1_onehot_test, catboost2_onehot_test,
                        catboost3_onehot_test, catboost4_onehot_test,
                        gru1_onehot_test, gru2_onehot_test])
#                         transformer1_onehot_test, transformer2_onehot_test])

print(train_final.shape)
print(test_final.shape)

# https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221343373342&proxyReferer=https:%2F%2Fwww.google.com%2F

(3125, 10)
(782, 10)


In [19]:
# 최종 앙상블

def ensemble_xgb(split_num, train, target, test):

    test_pred = np.zeros((test.shape[0], 61))
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 12,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                }
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=2021)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]
        
        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
    
    sample_submssion = pd.read_csv(path + 'sample_submission.csv')
    sample_submssion.iloc[:,1:] = test_pred
    sample_submssion.to_csv("ensemble.csv", index = False)
    
    print(sample_submssion)
    
ensemble_xgb(5, train_final, np.array(y_train), test_final)

[0]	train-mlogloss:3.55150	valid-mlogloss:3.55670
[100]	train-mlogloss:0.76089	valid-mlogloss:1.13455
[200]	train-mlogloss:0.57957	valid-mlogloss:1.11274
[216]	train-mlogloss:0.56618	valid-mlogloss:1.11539
------------------
[0]	train-mlogloss:3.55236	valid-mlogloss:3.55924
[100]	train-mlogloss:0.75438	valid-mlogloss:1.16385
[200]	train-mlogloss:0.57494	valid-mlogloss:1.14806
[203]	train-mlogloss:0.57227	valid-mlogloss:1.14872
------------------
[0]	train-mlogloss:3.55097	valid-mlogloss:3.56273
[100]	train-mlogloss:0.76071	valid-mlogloss:1.12269
[196]	train-mlogloss:0.58368	valid-mlogloss:1.11362
------------------
[0]	train-mlogloss:3.55249	valid-mlogloss:3.55610
[100]	train-mlogloss:0.77599	valid-mlogloss:1.05453
[200]	train-mlogloss:0.59372	valid-mlogloss:1.01281
[240]	train-mlogloss:0.56373	valid-mlogloss:1.01914
------------------
[0]	train-mlogloss:3.55495	valid-mlogloss:3.56369
[100]	train-mlogloss:0.77711	valid-mlogloss:1.06471
[188]	train-mlogloss:0.60903	valid-mlogloss:1.0494